In [ ]:
# Notebook Configuration
GPU_COUNT = 1
WINDOWS = True

In [ ]:
!pip install -q pandas
!pip install -q seaborn
!pip install -q matplotlib
!pip install -q neptune-client

<img src="images/cifar10.PNG" width=700>

# Generative Adversarial Networks - CIFAR10
> Can we create a strong generative model for CIFAR10 using the GAN architecture?

## Background Research 📖

### Introduction 💡
The ultimate goal of Deep Learning is to be able to create a function that can <strong>effectively</strong> model any form of data distribution. History has time and time again displayed the impressive success of discriminators, models that learn to divide the data distribution/map a high dimensional vector to one that is lower (Goodfellow et al., 2014). For instance, tasks such as Image Classification are one type of discriminative modelling, as the high dimensional images are mapped into low dimensional probabilities of labels.

What about generative modelling? In generative modelling, the goal is instead given a data distribution to learn from, to produce or generate new examples that follow but this distribution but still prove to be unique. Thus, a high performing generative model should be able to create examples that are both plausible (in that one can recognize what the generated example is supposed to be of) and indistinguishable from real data examples (Brownlee, 2019). Generative models can be Unsupervised or Semi-Supervised, depending on the exact task that one is trying to tackle. 

There are different approaches to network architectures when it comes to trying to achieve Generative Models:
<ul>
	<li>Generative Adversarial Networks (GAN) </li>
	<li>Diffusion Models</li>
	<li>Variational Auto Encoders (VAE)</li>
</ul>

<strong>GANs ⚔️</strong> <br />
GANs are the main focus of this notebook. Proposed by Ian Goodfellow in 2014, it became one of the more popular types of Generative Models used. For instance, the commonly known website <a href="thispersondoesnotexist.com">thispersondoesnotexist.com</a> uses the StyleGAN2 architecture (Karras et al., 2020). The idea for GANs is that there are two networks that work against each other in a game, where they try to one up each other. Thus, this leads to improvement in both networks. More details are discussed under "What's inside a GAN? 🔍".

<strong>Diffusion Models ✨</strong><br /> 
These are the models that have been not only been successful, but widely popular as well. For instance, OpenAI Dall-E, Google Imagen, Stable Diffusion, Midjourney are models that fall under the category of Diffusion Models. (Muppalla and Hendryx, 2022). From a high level, it works like so:
<ul>
	<li>Noise is added to original images</li>
	<li>Noise is procedurally added until image is all noise</li>
	<li>The model then learns to remove the noise</li>
	<li>Guidance can be added in the form of e.g. text-to-image, to provide direction of the generation process</li>
</ul>

<img src="images/diffusion1.jpg" width=400/><br />
*Noise is procedurally added to the image*

<img src="images/diffusion2.jpg" width=400/><br />
*Model attempts to recreate the image. (Muppalla and Hendryx, 2022)*



<strong>Variational Auto Encoders 🎲</strong><br />
We first take a look at what Auto Encoders are. Essentially,

Variational now induce randomness into these AutoEncoders.

### What's inside a GAN? 🔍
Generator and Discriminatorto use the well understood and established classification model to train 

### Types of GANs 🍐
GAN
Conditional GAN.

One may argue that the Conditional GAN.

### The difficulty with GANs 🧩
GANs are 
 
Mode collapse
Stability - hyperparamter tuning is very important versus discriminator models where hyperparameters more often than not determine .

### Loss Functions 🏓
Different Loss functions, main one being KL Divergence.
Explain how KL Divergence works.
There are papers claiming that novel loss functions improve stability.
There are also papers https://arxiv.org/abs/1811.09567 that claim that loss functions don't really matter.
This is an area of research I will attempt to explore in the notebook as well.

## Developing the GAN
### Objectives
<ol>
	<li>Explore the CIFAR10 dataset</li>
	<li>Implement and evaluate to find the best performing model</li>
	<li>Analyse the final model</li>
</ol>

### Importing Libraries

In [ ]:
import torch

### Utility Functions
We define some utility functions below that will ease and help us with our analysis.

In [ ]:
def loc_data(data, loc):
	datacopy = copy.deepcopy(data)
	arr = np.array(datacopy.loc[loc].drop('label'))
	label = datacopy.loc[loc]['label']
	root = int(len(arr) ** 0.5)
	arr.resize((root, root))
	return label, arr

def imshow(arr: list, label: list = None, figsize=None, shape = (32, 32, 3), is_int = None):
	if is_int == None:
		if type(arr[0]) == torch.Tensor:
			is_int = (arr[0].detach().cpu().numpy() > 1).sum() > 0
		else:
			is_int = (arr[0] > 1).sum() > 0
	if label == None:
		label = [''] * len(arr)

	height = int(len(arr) ** 0.5)
	width = math.ceil(len(arr) / height)

	if figsize == None:
		fig = plt.figure()
	else:
		fig = plt.figure(figsize=figsize)
	for i in range(height):
		for j in range(width):
			ax = fig.add_subplot(height, width, i * height + j + 1)
			ax.grid(False)
			ax.set_xticks([])
			ax.set_yticks([])
			show = arr[i * height + j]
			if type(arr[i * height + j]) != torch.Tensor:
				show = torch.Tensor(show)
				# ax.imshow((arr[i * height + j].squeeze(0).cpu().permute(1, 2, 0) / 255).type(torch.uint8 if is_int else float))
			# if (show.shape[0] == 1):
			# 	ax.imshow((show.squeeze(0).cpu()).type(torch.uint8 if is_int else torch.float), cmap='gray')
			# else:
			if len(show.squeeze(0).cpu().shape) == 2:
				ax.imshow((show.squeeze(0).detach().cpu()).type(torch.uint8 if is_int else torch.float), cmap='gray')
			else:
				ax.imshow((show.squeeze(0).detach().cpu().permute(1,2,0)).type(torch.uint8 if is_int else torch.float))
			ax.set_title(label[i * height + j])

def df_to_tensor(df, shape = (28, 28)):
	return torch.tensor(df.values.reshape((-1, *shape)), dtype=torch.float32)

def preprocess(df):
	return df.copy() / 255